In [8]:
import re
from src.model.synthesizer import InferenceConfig

from hparams import get_hparams_from_file
from src.text.test_sentences import kur_vasara_amzina_sentences, witty_sentences
%matplotlib inline

# def synthesize_filelist(config: InferenceConfig, number_of_entries=None):
#     test_set = TestSet.from_in_out_pair(InOutPair(in_filepath=config.filelist_path, out_directory=config.output_dir))
#     sentences = [entry.text for entry in test_set.entries] if config.stressed else [entry.remove_text_accents() for entry in
#                                                                              test_set.entries]
#
#     indices = list(range(0, len(sentences)))
#     if number_of_entries:
#         random.shuffle(indices)
#         indices = indices[:number_of_entries]
#         sentences = [sentences[idx] for idx in indices]
#     entries = [test_set.entries[idx] for idx in indices]
#
#     synthesize_all(
#         config.net_g,
#         sentences=sentences,
#         hps=config.hps,
#         plot_align=False,
#         verbose=False,
#         show_player=False,
#         output_dir=test_set.in_out_pair.out_directory)
#
#     save_output_filelist(entries, out_filepath=test_set.in_out_pair.out_directory / test_set.in_out_pair.in_filepath.name)
#
# def save_output_filelist(entries, out_filepath):
#     entry_lines = [f"{entry.path.name}|{normalize_sentence_to_filename(entry.text)}|{entry.text}|{entry.remove_text_accents()}" for entry in entries]
#
#     text = '\n'.join(entry_lines)
#     with open(out_filepath, mode='w', encoding='utf-8') as f:
#         f.write(text)

## InoVoice TTS

In [9]:
# filelist_dir = Path("/media/arnas/SSD Disk/uni/semester_4/thesis-files/mos/lt_synthesizer_survey")
# output_dir = Path("/media/arnas/SSD Disk/uni/semester_3/master_thesis/repos/lithuanian_synthesis_samples/files/audio/vits")
configs = [
    # InferenceConfig(
    #     checkpoint_step=150000,
    #     hps=hps22,
    #     speaker='aurimas_nausedas',
    #     stressed=False,
    #     filelist_path=filelist_dir / 'vits_aurimas_ns.txt',
    #     output_dir=output_dir / 'vits_aurimas_ns'
    # ),
    # InferenceConfig(
    #     checkpoint_step=150000,
    #     hps=hps22,
    #     speaker='aurimas_nausedas',
    #     stressed=True,
    #     filelist_path=filelist_dir / 'filelist_stressed.txt',
    #     output_dir=output_dir / 'vits_aurimas_s'
    # ),
    # InferenceConfig(
    #     checkpoint_step=1305000,
    #     hps=hps22,
    #     speaker='giedrius_altoriu_sesely',
    #     stressed=False,
    #     filelist_path=filelist_dir / 'vits_giedrius_ns.txt',
    #     output_dir=output_dir / 'vits_giedrius_ns'
    # ),
    # InferenceConfig(
    #     checkpoint_step=1305000,
    #     hps=hps22,
    #     speaker='giedrius_altoriu_sesely',
    #     stressed=True,
    #     filelist_path=filelist_dir / 'vits_giedrius_s.txt',
    #     output_dir=output_dir / 'vits_giedrius_s'
    # ),
    # InferenceConfig(
    #     checkpoint_step=3020000,
    #     hps=hps22,
    #     speaker='vytautas',
    #     stressed=False,
    #     filelist_path=filelist_dir / 'vits_vytautas_ns.txt',
    #     output_dir=output_dir / 'vits_vytautas_ns'
    # ),
    # InferenceConfig(
    #     checkpoint_step=3020000,
    #     hps=hps22,
    #     speaker='vytautas',
    #     stressed=True,
    #     filelist_path=filelist_dir / 'vits_vytautas_s.txt',
    #     output_dir=output_dir / 'vits_vytautas_s'
    # ),
    InferenceConfig(
        checkpoint_step=330000,
        config_name='44khz',
        speaker='aurimas_nausedas_44khz',
        stressed=True,
    ),
    InferenceConfig(
        checkpoint_step=365000,
        config_name='44khz',
        speaker='giedrius_altoriu_sesely_44khz',
        stressed=True,
    ),
]

[2022-06-22 16:28:03,420] INFO:PID-41706:MainThread:src.model.checkpoint: Loading checkpoint at /media/arnas/SSD Disk/inovoice/models/text-to-speech/vits/aurimas_nausedas_44khz/G_330000.pth


[INFO] Loading checkpoint at /media/arnas/SSD Disk/inovoice/models/text-to-speech/vits/aurimas_nausedas_44khz/G_330000.pth


[2022-06-22 16:28:04,360] INFO:PID-41706:MainThread:src.model.checkpoint: Loaded checkpoint '/media/arnas/SSD Disk/inovoice/models/text-to-speech/vits/aurimas_nausedas_44khz/G_330000.pth' (iteration 2143)


[INFO] Loaded checkpoint '/media/arnas/SSD Disk/inovoice/models/text-to-speech/vits/aurimas_nausedas_44khz/G_330000.pth' (iteration 2143)


[2022-06-22 16:28:04,839] INFO:PID-41706:MainThread:src.model.checkpoint: Loading checkpoint at /media/arnas/SSD Disk/inovoice/models/text-to-speech/vits/giedrius_altoriu_sesely_44khz/G_365000.pth


[INFO] Loading checkpoint at /media/arnas/SSD Disk/inovoice/models/text-to-speech/vits/giedrius_altoriu_sesely_44khz/G_365000.pth


[2022-06-22 16:28:06,357] INFO:PID-41706:MainThread:src.model.checkpoint: Loaded checkpoint '/media/arnas/SSD Disk/inovoice/models/text-to-speech/vits/giedrius_altoriu_sesely_44khz/G_365000.pth' (iteration 2135)


[INFO] Loaded checkpoint '/media/arnas/SSD Disk/inovoice/models/text-to-speech/vits/giedrius_altoriu_sesely_44khz/G_365000.pth' (iteration 2135)


In [10]:
sents = [
    "- Jū̃s klaũsote sintezúoto tèksto.",
    "Vìlniaus universitèto Matemãtikos ir̃ informãtikos fakultètas.",
    "Vãsara paskutìnių kùrsų studeñtams kasmẽt prasìdeda įtemptaĩ - láukia baigiamų̃jų darbų̃ gynìmai. Vìlniaus universitèto Matemãtikos ir̃ informãtikos fakultetè šiaĩs mẽtais jiẽ prasìdeda gegužė̃s trisdešimt pirmąją ir̃ baĩgsis biržẽlio dešimtą diẽną.",
]
sents += witty_sentences + kur_vasara_amzina_sentences

In [11]:
sentences = [re.sub('[\u0300\u0301\u0303]', '', s) for s in sents] if not configs[0].stressed else sents.copy()

In [12]:
for config in configs:
    config.synthesizer.synthesize_all(sentences, plot_align=False, output_dir=config.output_dir)


[2022-06-22 16:28:08,920] INFO:PID-41706:MainThread:src.model.synthesizer: Synthesized 21 utterances.


[INFO] Synthesized 21 utterances.


[2022-06-22 16:28:11,185] INFO:PID-41706:MainThread:src.model.synthesizer: Synthesized 21 utterances.


[INFO] Synthesized 21 utterances.
